In [51]:
file = "0050.csv"


In [52]:
import pandas as pd
import numpy as np


ticker = pd.read_csv(file)

# change the name of columns
ticker.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# set the column "Date" to index
ticker = ticker.set_index('Date')

# set index column("Date") from string to datetime.datetime
ticker.index = pd.to_datetime(ticker.index)

# sort index column("Date") chronologically
ticker = ticker.sort_index()

# take latest 300 price data for analysis
ticker = ticker.tail(300)
print(ticker)

              Open    High     Low   Close  Volume
Date                                              
2019-12-27   97.60   98.15   97.60   98.00    2978
2019-12-30   98.00   98.25   97.60   97.80    3138
2019-12-31   97.10   97.20   96.95   96.95    4303
2020-01-02   97.05   98.00   97.05   97.65    4882
2020-01-03   98.30   98.70   97.00   97.65    6813
...            ...     ...     ...     ...     ...
2021-03-22  132.40  133.60  131.60  133.40    5510
2021-03-23  133.90  134.70  133.60  133.60    6994
2021-03-24  131.80  132.45  131.30  131.50   11082
2021-03-25  131.10  132.55  130.60  131.95    5186
2021-03-26  133.05  134.40  132.55  134.35    8134

[300 rows x 5 columns]


In [53]:
# Define constants
RSI_n = 6
upperBound = 90
lowerBound = 10
middle_line = 50

# Create a empty dataFrame with original index
# To store some temporary data
tmp = pd.DataFrame(index=ticker.index)

# Copy a dataFrame to store result
# deep=True means make a copy instead of link
data = ticker.copy(deep=True)

# tmp['rise'] = ticker['Close'] - ticker['Open']

tmp['rise'] = np.nan
rsi = []
for i in range(1, len(ticker['Close'])):
    rsi.append(round(ticker['Close'][i] - ticker['Close'][i - 1], 2))
tmp['rise'][1:] = rsi.copy()

tmp['rise_only'] = tmp['rise']
tmp['drop_only'] = tmp['rise']

tmp['rise_only'].loc[tmp['rise'] < 0] = 0
tmp['drop_only'].loc[tmp['rise'] > 0] = 0
print(tmp.head(10))


tmp['rise_avg'] = tmp['rise_only'].rolling(RSI_n).mean()
tmp['drop_avg'] = tmp['drop_only'].rolling(RSI_n).mean().abs()
print(tmp.head(10))

data['RSI'] = tmp['rise_avg'] / (tmp['rise_avg'] + tmp['drop_avg']) * 100
print(data.head(10))

            rise  rise_only  drop_only
Date                                  
2019-12-27   NaN        NaN        NaN
2019-12-30 -0.20       0.00      -0.20
2019-12-31 -0.85       0.00      -0.85
2020-01-02  0.70       0.70       0.00
2020-01-03  0.00       0.00       0.00
2020-01-06 -1.25       0.00      -1.25
2020-01-07 -0.30       0.00      -0.30
2020-01-08 -0.45       0.00      -0.45
2020-01-09  1.30       1.30       0.00
2020-01-10  0.35       0.35       0.00
            rise  rise_only  drop_only  rise_avg  drop_avg
Date                                                      
2019-12-27   NaN        NaN        NaN       NaN       NaN
2019-12-30 -0.20       0.00      -0.20       NaN       NaN
2019-12-31 -0.85       0.00      -0.85       NaN       NaN
2020-01-02  0.70       0.70       0.00       NaN       NaN
2020-01-03  0.00       0.00       0.00       NaN       NaN
2020-01-06 -1.25       0.00      -1.25       NaN       NaN
2020-01-07 -0.30       0.00      -0.30  0.116667  0.433333
2

In [54]:
# Define constants
KD_n = 9
upperBound = 80
lowerBound = 20
middle_line = 50

# Create a empty dataFrame with original index
# To store some temporary data
tmp = pd.DataFrame(index=ticker.index)

# Copy a dataFrame to store result
# deep=True means make a copy instead of link
data = ticker.copy(deep=True)

tmp['max_close'] = ticker['Close'].rolling(KD_n, min_periods=1).max()
tmp['min_close'] = ticker['Close'].rolling(KD_n, min_periods=1).min()

# rolling() 的用法可參考 https://vimsky.com/zh-tw/examples/usage/python-pandas-dataframe-rolling.html (modified: Lai)

# RSV also called FastK
tmp['RSV'] = (ticker['Close'] - tmp['min_close'])/(tmp['max_close']-tmp['min_close'])*100
print(tmp.head(30))

data['k'] = np.zeros(ticker.shape[0])
data['d'] = np.zeros(ticker.shape[0])

# skip first day
# because RSV of first day is NaN
print(data.head(30))
for i in range(1, tmp.shape[0]):
    
    # means data['k'][i]
    data.loc[(data.index[i], 'k')] = data['k'][i-1] * (2/3) + tmp['RSV'][i] * (1/3)
    data.loc[(data.index[i], 'd')] = data['d'][i-1] * (2/3) + data['k'][i] * (1/3)

print(data.head(30))

            max_close  min_close         RSV
Date                                        
2019-12-27      98.00      98.00         NaN
2019-12-30      98.00      97.80    0.000000
2019-12-31      98.00      96.95    0.000000
2020-01-02      98.00      96.95   66.666667
2020-01-03      98.00      96.95   66.666667
2020-01-06      98.00      96.40    0.000000
2020-01-07      98.00      96.10    0.000000
2020-01-08      98.00      95.65    0.000000
2020-01-09      98.00      95.65   55.319149
2020-01-10      97.80      95.65   76.744186
2020-01-13      98.00      95.65  100.000000
2020-01-14      98.60      95.65  100.000000
2020-01-15      98.60      95.65   66.101695
2020-01-16      98.60      95.65   55.932203
2020-01-17      98.60      95.65   55.932203
2020-01-20      98.60      95.65   69.491525
2020-01-30      98.60      92.15    0.000000
2020-01-31      98.60      89.95    0.000000
2020-02-03      98.60      89.05    0.000000
2020-02-04      98.60      89.05   16.230366
2020-02-05

In [55]:
# Define constants
upperBound = 80
lowerBound = 20
middle_line = 50

def KD(_ticker, n_days = 9):

    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=_ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    res = _ticker.copy(deep=True)

    tmp['max_close'] = ticker['Close'].rolling(n_days).max()
    tmp['min_close'] = ticker['Close'].rolling(n_days).min()

    # RSV also called FastK
    tmp['RSV'] = (ticker['Close'] - tmp['min_close']) / (tmp['max_close'] - tmp['min_close']) * 100
    print(tmp.head(30))
    res['K'] = 0
    res['D'] = 0
    col_k = res.columns.get_loc('K')
    col_d = res.columns.get_loc('D')
    col_rsv = tmp.columns.get_loc('RSV')
    print(res.head(30))

    for i in range(9, len(_ticker)):
        res.iloc[i, col_k] = res.iloc[i - 1, col_k] * 2 / 3 + tmp.iloc[i, col_rsv] * 1 / 3
        res.iloc[i, col_d] = res.iloc[i - 1, col_d] * 2 / 3 + res.iloc[i, col_k] * 1 / 3
    
    print(res.head(30))
    return res

In [56]:
import pandas as pd
import numpy as np


ticker = pd.read_csv(file)

# change the name of columns
ticker.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# set the column "Date" to index
ticker = ticker.set_index('Date')

# set index column("Date") from string to datetime.datetime
ticker.index = pd.to_datetime(ticker.index)

# sort index column("Date") chronologically
ticker = ticker.sort_index()

# take latest 300 price data for analysis
ticker = ticker.tail(300)
print(ticker)

ticker = KD(ticker)
print(ticker)

              Open    High     Low   Close  Volume
Date                                              
2019-12-27   97.60   98.15   97.60   98.00    2978
2019-12-30   98.00   98.25   97.60   97.80    3138
2019-12-31   97.10   97.20   96.95   96.95    4303
2020-01-02   97.05   98.00   97.05   97.65    4882
2020-01-03   98.30   98.70   97.00   97.65    6813
...            ...     ...     ...     ...     ...
2021-03-22  132.40  133.60  131.60  133.40    5510
2021-03-23  133.90  134.70  133.60  133.60    6994
2021-03-24  131.80  132.45  131.30  131.50   11082
2021-03-25  131.10  132.55  130.60  131.95    5186
2021-03-26  133.05  134.40  132.55  134.35    8134

[300 rows x 5 columns]
            max_close  min_close         RSV
Date                                        
2019-12-27        NaN        NaN         NaN
2019-12-30        NaN        NaN         NaN
2019-12-31        NaN        NaN         NaN
2020-01-02        NaN        NaN         NaN
2020-01-03        NaN        NaN         Na